# Attendance Tracking System - Jupyter Notebook

## Introduction
The Attendance Tracking System automates the process of tracking student attendance based on their device connections. This notebook will explain the implementation of the system step by step.

## Step 1: Imports
First, we import the necessary modules for our project.

In [ ]:
python
import json
from flask import Flask, render_template
from datetime import datetime

## Step 2: Define Student Class
We define a `Student` class to represent student objects. Each student has attributes such as name, roll number, and MAC ID.

In [ ]:

class Student:
    def __init__(self, name, roll_number, mac_id):
        self.name = name
        self.roll_number = roll_number
        self.mac_id = mac_id

    def to_dict(self):
        return {
            'name': self.name,
            'roll_number': self.roll_number,
            'mac_id': self.mac_id
        }


## Step 3: Load Student Data
We create a function `load_students` to load student data from a JSON file (`students.json`). The function returns a list of `Student` objects.


In [ ]:
def load_students(filename):
    try:
        with open(filename, 'r') as f:
            data = json.load(f)
            student_data = data.get("students", [])
            students = [Student(student['name'], student['roll_number'], student['mac_id']) for student in student_data]
        print("Student details loaded successfully")
        return students
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found")
    except Exception as e:
        print("Error loading student details:", e)
        return None

## Step 4: Get Connected Devices
We create a function `get_connected_devices` to simulate retrieving connected devices. For demonstration purposes, we return a dummy list of connected devices.

In [ ]:
def get_connected_devices():
    print("Step 1: Getting connected devices...")
    connected_devices = ['6c:03:b5:94:9f:f2']  # Dummy data for demonstration
    print("Connected devices retrieved")
    return connected_devices

## Step 5: Save Device Details
We create a function `save_device_details` to save the list of connected devices to a JSON file (`devices.json`).


In [ ]:
def save_device_details(devices):
    print("Step 2: Saving device details...")
    try:
        with open('devices.json', 'w') as f:
            json.dump(devices, f)
        print("Device details saved to 'devices.json'")
    except Exception as e:
        print("Error saving device details:", e)

## Step 6: Verify Attendance
We create a function `verify_attendance` to verify attendance based on connected devices and student data. We match the MAC IDs of connected devices with the MAC IDs of students and record the attendance accordingly.

In [ ]:
def verify_attendance(students, devices):
    print("Step 3: Verifying attendance...")
    if students is None:
        print("Cannot verify attendance. Student details not loaded.")
        return
    
    present_students = []
    absent_students = []
    
    for student in students:
        if student.mac_id in devices:
            connection_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            student_info = {
                'name': student.name,
                'roll_number': student.roll_number,
                'mac_id': student.mac_id,
                'connection_time': connection_time
            }
            present_students.append(student_info)
        else:
            absent_students.append(student)
                
    with open('present.json', 'w') as f:
        json.dump(present_students, f, default=str)
    print("Present students saved to 'present.json'")
    
    with open('absent.json', 'w') as f:
        json.dump([s.__dict__ for s in absent_students], f, default=str)
    print("Absent students saved to 'absent.json'")

## Step 7: Flask Application
We define a Flask application to handle HTTP requests and render HTML templates. The main route (`/`) loads student data, retrieves connected devices, and verifies attendance.


In [ ]:
app = Flask(__name__)

@app.route('/')
def index():
    students = load_students('students.json')
    connected_devices = get_connected_devices()
    
    if students is not None and connected_devices is not None:
        verify_attendance(students, connected_devices)
    else:
        print("Cannot verify attendance. Some data is missing.")
    
    with open('present.json', 'r') as f:
        present_data = json.load(f)
    with open('absent.json', 'r') as f:
        absent_data = json.load(f)

    return render_template('index.html', present_data=present_data, absent_data=absent_data)

if __name__ == '__main__':
    app.run(debug